In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import json

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


## Check in 

In [7]:
checkinfile = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json")
data = []
for line in checkinfile:
    data.append(json.loads(line))
checkin = pd.DataFrame(data)
checkinfile.close()

In [8]:
checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [39]:
earliest_date = str(checkin_data['date'].min())[:10]
latest_date = checkin_data['date'].max()

print("Check-in table earliest date:", earliest_date)
print("Check-in table latest date:", latest_date)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


## Review

In [9]:
reviewfile = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json")
data = []
for line in reviewfile:
    data.append(json.loads(line))
review = pd.DataFrame(data)
reviewfile.close()

In [10]:
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [ ]:
earliest_date = review['date'].min()
latest_date = review['date'].max()

print("Review table earliest date:", earliest_date)
print("Review table latest date:", latest_date)

## Business

In [11]:
businessfile = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json")
data = []
for line in businessfile:
    data.append(json.loads(line))
business = pd.DataFrame(data)
businessfile.close()

In [54]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [ ]:
if 'attributes' in business.columns:
    business['attributes'] = business['attributes'].astype(str)
else:
    print("Warning: 'attributes' column not found in the data.")

if 'hours' in business.columns:
    business['hours'] = business['hours'].astype(str)
else:
    print("Warning: 'hours' column not found in the data.")

## Tip

In [13]:
tipfile = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json")
data = []
for line in tipfile:
    data.append(json.loads(line))
tip = pd.DataFrame(data)
tipfile.close()

In [14]:
tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [ ]:
earliest_date = tip['date'].min()
latest_date = tip['date'].max()

print("Tip table earliest date:", earliest_date)
print("Tip table latest date:", latest_date)

## User

In [18]:
userfile = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json")
data = []
for line in userfile:
    data.append(json.loads(line))
user = pd.DataFrame(data)
userfile.close()

In [33]:
user.head()
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 333.7+ MB


In [ ]:
earliest_date = user['yelping_since'].min()
latest_date = user['yelping_since'].max()

print("User table earliest date:", earliest_date)
print("User table latest date:", latest_date)

# Connecting to Azure SQL Server

In [77]:
from sqlalchemy import create_engine, types
import sqlalchemy
!pip install PyMySql
!pip install pyodbc

In [84]:
!apt-get update
!apt-get install -y unixodbc unixodbc-dev

# Add Microsoft's signing key and repository for SQL Server ODBC Driver
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | tee /etc/apt/sources.list.d/mssql-release.list

# Update the package list again after adding Microsoft's repository
!apt-get update

# Install Microsoft SQL Server ODBC Driver 17
!ACCEPT_EULA=Y apt-get install -y msodbcsql17

Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]       
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:5 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [28.6 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1564 kB]
Get:8 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3375 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1275 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4258 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4105 kB]
Get:13 ht

## Connection Details

In [85]:
username = 'yh69-admin'
password = 'nimda-96hy'
server = 'yh69-server.database.windows.net'
database = 'badm554minicase1'

## Setup Connection

In [98]:
connection_string = f"mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)
print("connection success!")

connection success!


In [105]:
try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print("Failed to connect:", e)

Connection successful!


## RUN QUERY

In [ ]:
business.to_sql(
    name="review",
    con=engine,
    if_exists='replace',
    index=True,
    dtype={'attributes': sqlalchemy.types.NVARCHAR, 'hours': sqlalchemy.types.NVARCHAR},
    chunksize=1000
)